# Lieux de tournage à paris

In [57]:
import pandas as pd
import numpy as np
df_tournage = pd.read_csv("src/lieux-de-tournage-a-paris.csv", delimiter=';', on_bad_lines='skip')

# Drop unused :
df_tournage = df_tournage.drop(columns=["Identifiant du lieu","Localisation de la scène","Année du tournage","Coordonnée en X", "Coordonnée en Y","geo_shape"])

df_tournage.dtypes

Type de tournage     object
Titre                object
Réalisateur          object
Producteur           object
Code postal         float64
Date de début        object
Date de fin          object
geo_point_2d         object
dtype: object

In [58]:
# Corrections Type :
df_tournage["Code postal"] = pd.to_numeric(df_tournage["Code postal"])
df_tournage = df_tournage.dropna(subset = ['Code postal'])
df_tournage["Code postal"] = df_tournage["Code postal"].astype(int)
df_tournage["Date de début"] = pd.to_datetime(df_tournage["Date de début"])
df_tournage["Date de fin"] = pd.to_datetime(df_tournage["Date de fin"])
df_tournage["latitude"] = pd.to_numeric( [ x.split(",")[0] for x in df_tournage["geo_point_2d"][:][:] ] )
df_tournage["longitude"] = pd.to_numeric( [ x.split(",")[1] for x in df_tournage["geo_point_2d"][:][:] ] )
df_tournage = df_tournage.drop(columns="geo_point_2d")

# Clear Name Columns :
df_tournage = df_tournage.rename(columns = {"Type de tournage":"type", "Code postal":"postal", "Date de début":"debut", "Date de fin":"fin"})
df_tournage.columns = [ x.lower() for x in df_tournage.columns.tolist() ] # ABC -> abc

df_tournage.head()

,type,titre,réalisateur,producteur,postal,debut,fin,latitude,longitude
0,Long métrage,TOUT S'EST BIEN PASSE,Francois OZON,MANDARIN PRODUCTION,75013,2020-08-20,2020-08-21,48.835660,2.348315
1,Série Web,Ca ne va pas Supermarché,Pierre Briois et Théo Galampoix,CLCF (Conservatoire Libre du Cinéma Français),75014,2020-08-23,2020-08-23,48.834767,2.315523
2,Long métrage,Une jeune fille qui va bien,Sandrine Kiberlain,CURIOSA FILMS,75004,2020-08-31,2020-09-01,48.854533,2.361694
3,Téléfilm,BALTHAZAR 23 et 24,CAMILLE DELAMARRE ET FRANCK BRETT,BEAUBOURG STORIES,75019,2020-09-04,2020-09-04,48.882138,2.394658
4,Série TV,Alice NEVERS,julien ZIDI,EGO PRoductions,75019,2019-11-28,2019-11-28,48.874095,2.375253


In [59]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

# Univariate analysis
# Distribution of each numeric variable

# fig = px.histogram(df_tournage, x="type")
fig = px.pie(df_tournage, names='type')
fig.show()

In [60]:
fig = px.pie(df_tournage, names='postal')
fig.show()

In [61]:
len(df_tournage)

10765

In [62]:
rive_droite = [75001, 75002, 75003, 75004, 75008, 75009, 75010, 75011, 75012, 75016, 75017, 75018, 75019, 75020]
rive_gauche = [75005, 75006, 75007, 75013, 75014, 75015]

df_tournage["rive"] = df_tournage["postal"].apply(lambda x : "droite" if x in rive_droite else "gauche" if x in rive_gauche else "couronne" )

# df_tournage["rive"].unique()
# df_tournage["rive"].value_counts()

fig = px.histogram(df_tournage, x="rive", color="type")
fig.show()


In [63]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(
    rows=1, cols=3,
    specs=[[{"type": "domain"}, {"type": "domain"}, {"type": "domain"}]],
)

# px.pie(df_tournage, names='type')


fig.add_trace(go.Pie(values= df_tournage[df_tournage["rive"] == 'droite'].type.value_counts() ),
              row=1, col=1)

fig.add_trace(go.Pie(values= df_tournage[df_tournage["rive"] == 'gauche'].type.value_counts() ),
              row=1, col=2)

fig.add_trace(go.Pie(values= df_tournage[df_tournage["rive"] == 'couronne'].type.value_counts() ),
              row=1, col=3)

fig.update_layout(height=700, showlegend=True)

fig.show()

# https://plotly.com/python/subplots/

In [64]:
df_tournage.head()

,type,titre,réalisateur,producteur,postal,debut,fin,latitude,longitude,rive
0,Long métrage,TOUT S'EST BIEN PASSE,Francois OZON,MANDARIN PRODUCTION,75013,2020-08-20,2020-08-21,48.835660,2.348315,gauche
1,Série Web,Ca ne va pas Supermarché,Pierre Briois et Théo Galampoix,CLCF (Conservatoire Libre du Cinéma Français),75014,2020-08-23,2020-08-23,48.834767,2.315523,gauche
2,Long métrage,Une jeune fille qui va bien,Sandrine Kiberlain,CURIOSA FILMS,75004,2020-08-31,2020-09-01,48.854533,2.361694,droite
3,Téléfilm,BALTHAZAR 23 et 24,CAMILLE DELAMARRE ET FRANCK BRETT,BEAUBOURG STORIES,75019,2020-09-04,2020-09-04,48.882138,2.394658,droite
4,Série TV,Alice NEVERS,julien ZIDI,EGO PRoductions,75019,2019-11-28,2019-11-28,48.874095,2.375253,droite


In [65]:
import datetime as dt 

fig = px.scatter_mapbox(df_tournage, lat="latitude", lon="longitude", color="type", zoom = 10,
                        mapbox_style="open-street-map", color_continuous_scale = 'Reds', 
                       animation_frame = df_tournage["debut"].dt.year + df_tournage["debut"].dt.month/100 )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

# fig.write_html("file.html")

In [66]:
import datetime as dt 
duree = [ x + dt.timedelta(days= 1) for x in df_tournage["fin"] - df_tournage["debut"] ]
df_tournage["duree"] = [ x.days for x in duree ]

fig = px.scatter_mapbox(df_tournage[ df_tournage["duree"] < 4 ], 
                        lat="latitude", 
                        lon="longitude", 
                        color="duree", 
                        zoom = 11,
                        mapbox_style="open-street-map", 
                        color_continuous_scale = 'Reds')
                        
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [67]:
fig = px.density_mapbox(df_tournage[ df_tournage["duree"] < 4 ], 
                        lat="latitude", 
                        lon="longitude", 
                        radius=3, 
                        zoom=11,
                        mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [68]:
#df_tournage[df_tournage["rive"] == 'droite'].type.value_counts()

fig = go.Figure(
    data = go.Histogram(
        x = df_tournage['debut'], nbinsx = df_tournage['debut'].nunique()),
    layout = go.Layout(
        title = go.layout.Title(text = "Nombre de tournage par jour", x = 0.5),
        #xaxis = go.layout.XAxis(title = 'X', rangeslider = go.layout.xaxis.Rangeslider(visible = True))
    )
)

fig.show()

In [69]:
df_tournage.head()

,type,titre,réalisateur,producteur,postal,debut,fin,latitude,longitude,rive,duree
0,Long métrage,TOUT S'EST BIEN PASSE,Francois OZON,MANDARIN PRODUCTION,75013,2020-08-20,2020-08-21,48.835660,2.348315,gauche,2
1,Série Web,Ca ne va pas Supermarché,Pierre Briois et Théo Galampoix,CLCF (Conservatoire Libre du Cinéma Français),75014,2020-08-23,2020-08-23,48.834767,2.315523,gauche,1
2,Long métrage,Une jeune fille qui va bien,Sandrine Kiberlain,CURIOSA FILMS,75004,2020-08-31,2020-09-01,48.854533,2.361694,droite,2
3,Téléfilm,BALTHAZAR 23 et 24,CAMILLE DELAMARRE ET FRANCK BRETT,BEAUBOURG STORIES,75019,2020-09-04,2020-09-04,48.882138,2.394658,droite,1
4,Série TV,Alice NEVERS,julien ZIDI,EGO PRoductions,75019,2019-11-28,2019-11-28,48.874095,2.375253,droite,1


In [70]:
df_tournage_films = df_tournage[df_tournage["type"] == 'Long métrage']
df_tournage_series = df_tournage[df_tournage["type"] == 'Série TV']

df_tournage_films.head()

,type,titre,réalisateur,producteur,postal,debut,fin,latitude,longitude,rive,duree
0,Long métrage,TOUT S'EST BIEN PASSE,Francois OZON,MANDARIN PRODUCTION,75013,2020-08-20,2020-08-21,48.835660,2.348315,gauche,2
2,Long métrage,Une jeune fille qui va bien,Sandrine Kiberlain,CURIOSA FILMS,75004,2020-08-31,2020-09-01,48.854533,2.361694,droite,2
5,Long métrage,French Exit,Azazel Jacobs,Same Player,75012,2019-12-04,2019-12-04,48.850067,2.376519,droite,1
6,Long métrage,FIN DE MATINEE,Hiroshi NISHATANI,COMME DES CINEMAS,75004,2018-11-05,2018-11-05,48.854112,2.354679,droite,1
7,Long métrage,HORS NORMES,Eric Toledano et Olivier Nakache,ADNP QUAD FILMS,75001,2018-11-05,2018-11-06,48.865744,2.327446,droite,2


In [71]:
df_tournage_films = df_tournage_films.drop(columns=["type","duree"]).reset_index()
df_tournage_films = df_tournage_films.drop(columns=["index"])

In [76]:
df_tournage_films.to_csv(r'export/df_tournage_films.csv', index = False, header=True)